<a target="_blank" href="https://colab.research.google.com/github/pds2526/course/blob/main/06_Image%20Classification.ipynb
">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

<div class='bar_title'></div>

*Practical Data Science*

# Deep Learning for Computer Vision

Gunther Gust<br>
Chair for Enterprise AI<br>
Data Driven Decisions Group (D3)<br>
Center for Artificial Intelligence and Data Science (CAIDAS)

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/d3.png?raw=true" style="width:20%; float:left;" />

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/CAIDASlogo.png?raw=true" style="width:20%; float:left;" />

__Credits for this lecture__

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/09/oreillybook.jpg?raw=true" width="500" align="right"/>

**Jeremy Howard and Sylvian Gugger: "Deep Learning for Coders with Fastai and PyTorch: AI Applications without a PhD." (2020).**

Available as [Jupyter Notebook](https://github.com/fastai/fastbook)

Materials taken from
- https://github.com/fastai/fastbook/blob/master/05_pet_breeds.ipynb
- https://github.com/hiromis/notes/blob/master/Lesson1.md
- https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html

## Processing Image Data

### Requirements: Graphics Processing Unit (GPU)


GPU is fit for training the deep learning systems in a long run for very large datasets. CPU can train a deep learning model quite slowly. GPU accelerates the training of the model. Hence, GPU is a better choice to train the Deep Learning Model efficiently and effectively ([see here for a nice informal explanation](https://medium.com/@shachishah.ce/do-we-really-need-gpu-for-deep-learning-47042c02efe2)) (open in incognito tab to circument paywall).

Make sure your GPU environment is set up and you can run Jupyter Notebook.



#### GPU on Colab

* Select 'Runtime' -> 'Change runtime type' -> 'Python 3' and a 'GPU' or (Tensor Processing Unit, 'TPU') before running the notebook.

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/gpu_colab.png?raw=true" width="700" align="center"/>

### Libraries

We are going to work with the fastai V2 library which sits on top of Pytorch.
The fastai library as a [layered API](https://raw.githubusercontent.com/GuntherGust/tds2_data/refs/heads/main/images/09/fastai-api.webp) as summarized by this graph:

<img src="https://docs.fast.ai/images/layered.png" width="500"/>

### FastAI Architecture

#### Applications Layer
This is the topmost layer, offering the highest level of abstraction. It provides predefined applications for different types of data such as:
- `Vision` for image data
- `Text` for natural language processing
- `Tabular` for structured data like CSV files
- `Collab` which refers to collaborative filtering used in recommendation systems

#### High-Level API
Below the applications layer, there's the high-level API, which offers more control while still being relatively abstract. This includes:
- `Learner`: an interface for creating and training machine learning models
- `DataBlock`: an abstraction for defining how to load, split, and prepare data for modeling.


#### Mid-Level API
This layer provides more granular control over the training process, with components like:
- `Callbacks`: hooks for executing custom code at certain points in the training loop
- `Generic Optimizer`: for customizing the optimization algorithm
- `General Metric`: for customizing evaluation metrics
- `Data Core`: the fundamental data structures upon which the higher-level abstractions are built.

#### Low-Level API
The foundation that provides maximum flexibility and control, consisting of:
- `Pipeline`: a sequence of data processing operations
- `Reversible Transforms`: data transformations that can be applied and then reversed
- `OO Tensors`: object-oriented design for tensor operations
- `Optimized ops`: highly efficient, low-level operations.

On google colab, you may need to install or upgrade fastai

In [ ]:
#we need the current git version
#%pip install --upgrade fastai

import warnings
warnings.filterwarnings('ignore')


and import the library

In [ ]:
from fastai.vision.all import *

### Dataset: Imagewoof

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/09/dogs.jpg?raw=true" width="500" align="right"/>


We are going to use the [Imagewoof](https://github.com/fastai/imagenette) data set, a subset of 10 classes from Imagenet that aren't so easy to classify, since they're all dog breeds.

The breeds are: Australian terrier, Border terrier, Samoyed, Beagle, Shih-Tzu, English foxhound, Rhodesian ridgeback, Dingo, Golden retriever, Old English sheepdog.

__Download and extract__

The first thing we have to do is download and extract the data that we want. `untar_data` will download that to some convenient path and untar it for us and it will then return the value of path.

In [ ]:
# URLs.IMAGEWOOF_160 = 'https://s3.amazonaws.com/fast-ai-imageclas/imagewoof-160'
path = untar_data(URLs.IMAGEWOOF_160)
path

Next time you run this, since you've already downloaded it, it __won't download it again.__ Since you've already untared it, it won't untar it again. So everything is designed to be pretty __automatic and easy.__

### Looking at the data

The first thing we do when we approach a problem is to take a look at the data. We always need to understand very well what the problem is and what the data looks like before we can figure out how to solve it. Taking a look at the data means understanding how the data directories are structured, what the labels are and what some sample images look like.

__Python 3 pathlib__

For convenience, fast.ai adds functionality into existing Python stuff. One of these things is add a `ls()` method to path,  an IPython magic wrapper for the shell’s `ls`, showing the filesystem __contents__ of the current __directory.__

In [ ]:
# Lists the contents of the directory represented by 'path'
path.ls()

In [ ]:
# Lists the contents of the 'train' subdirectory in the directory represented by 'path'
(path/'train').ls()

### Mentimeter

> How is the downloaded data set of dog images structured? Please answer [on Mentimeter](https://www.menti.com/aleeov3mdgtg).

__Path objects__ from the [pathlib](https://docs.python.org/3/library/pathlib.html) module are much better to use than strings. It doesn't matter if you're on Windows, Linux, or Mac. It is always going to work exactly the __same way.__

__get_image_files__

`get_image_files` will just grab an array of all of the image files based on extension in a path.

In [ ]:
fnames = get_image_files(path/'train')
fnames

Here, 'n02115641' refers to the class _dingo_ in [imagenet](https://gist.github.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57).

In [ ]:
import imageio.v3 as imageio
# Displays the first image from the 'fnames'
plt.imshow(imageio.imread(fnames[10]));

### Creating DataLoaders

The main difference between the handling of image classification datasets is the way labels are stored. In this particular dataset, labels are stored in the __names of the (sub-)folders.__ We will need to extract them to be able to classify the images into the correct categories.

We will now explore different ways load such datasets.

#### Factory Methods: ImageDataLoaders

The [`ImageDataLoaders`](https://docs.fast.ai/vision.data.html#ImageDataLoaders) Class is a basic wrapper around several DataLoaders with factory methods for computer vision problems.

Here, we can use `ImageDataLoaders.from_folder`:

In [ ]:
# Creates image data loaders from folders, specifying 'train' and 'val' for training and validation sets, and applies resizing to 224x224 pixels for each item.
dls = ImageDataLoaders.from_folder(path, train='train', valid='val', item_tfms=Resize(224))

__data.show_batch__

Let's take a look at a few pictures. `dls.show_batch` can be used to show me some of the contents  So you can see roughly what's happened is that they all seem to have being zoomed and cropped in a reasonably nice way.

In [ ]:
dls.show_batch()

### The data block API

The [data block API](https://docs.fast.ai/data.block.html#DataBlock) lets you customize the creation of the Dataloaders by isolating the underlying parts of that process in separate blocks, mainly:

1. The types of your input and labels
2. `get_items` (how to get your input)
3. `splitter` (How to split the data into a training and validation sets?)
4. `get_y` (How to label the inputs?)

... and suitable transforms.

In [ ]:
woof = DataBlock(
    # Specifies the types of blocks: ImageBlock for the input images and CategoryBlock for the target labels (categories).
    blocks = (ImageBlock, CategoryBlock),
    # Sets the function to gather image files from directories
    get_items=get_image_files,
    # Uses GrandparentSplitter to split the dataset into training and validation sets based on folder names ('train' and 'val').
    splitter=GrandparentSplitter(train_name='train', valid_name='val'),
    # Sets the function to label the data items based on the parent folder name.
    get_y=parent_label,
    # Applies an item-wise transformation to resize each image to 460 pixels on the smallest side.
    item_tfms=Resize(460),
    # Applies batch-wise transformations for data augmentation, resizing images to 256 pixels and using a minimum scale of 0.75.
    batch_tfms=aug_transforms(size=256, min_scale=0.75)
)

One important piece of this `DataBlock` call that we haven't seen before is in these two lines:

```python
item_tfms=Resize(460),
batch_tfms=aug_transforms(size=224, min_scale=0.75)
```

These lines implement a fastai data augmentation strategy which we call __*presizing*.__ Presizing is a particular way to do image augmentation that is designed to minimize data destruction while maintaining good performance. When an image is randomly cropped as part of augmentation, the crop's size is a random value greater than or equal to min_scale times the size of the original image. For instance, if min_scale is set to 0.75, the random crop will be __at least 75%__ of the original image's size.

#### Presizing

<img alt="Presizing on the training set" width="600" caption="Presizing on the training set" id="presizing" src="https://github.com/GuntherGust/tds2_data/blob/main/images/09/fastai-cropping.png?raw=true" align="right">

Presizing adopts two strategies

1. *Crop full width or height*: This is in `item_tfms`, so it's applied to each individual image before it is copied to the GPU. It's used to ensure all images are the same size. On the training set, the crop area is chosen randomly. On the validation set, the center square of the image is always chosen.
2. *Random crop and augment*: This is in `batch_tfms`, so it's applied to a batch all at once on the GPU, which means it's fast. On the validation set, only the resize to the final size needed for the model is done here. On the training set, the random crop and any other augmentations are done first.

To implement this process in fastai you use `Resize` as an item transform with a large size, and `RandomResizedCrop` as a batch transform with a smaller size. `RandomResizedCrop` will be added for you if you include the `min_scale` parameter in your `aug_transforms` function, as was done in the `DataBlock` call in the previous section. Alternatively, you can use `pad` or `squish` instead of `crop` (the default) for the initial `Resize`.


The **difference between Crop, Pad, and Squish** in the Resize transform in FastAI lies in how the image is adjusted to fit the specified size:

1. **Crop (Default)**:   The image is resized while maintaining its aspect ratio, and then a central or random crop is taken to match the target size.  
   - **Effect**: Parts of the image outside the crop area are discarded.  
   - **Use Case**: When keeping the aspect ratio matters, and you don't mind losing some image content.

2. **Pad**:   The image is resized while maintaining its aspect ratio, then padding is added to match the target size.  
   - **Effect**: No parts of the image are lost, but padding (usually black) is added, potentially introducing unwanted artifacts.  
   - **Use Case**: When keeping the entire image without distortion is important.

3. **Squish**:  The image is resized to fit the target size without maintaining the aspect ratio.  
   - **Effect**: The image is stretched or compressed, possibly distorting its appearance.  
   - **Use Case**: When retaining the full image content is critical, and distortion is acceptable.


#### Dataloaders

From the Datablock we can automatically get a our DataLoaders:

In [ ]:
dls = woof.dataloaders(path)

.. and have a look at the summary:

In [ ]:
woof.summary(path)

In [ ]:
dls.show_batch()

To make the classes easier to read and interpret, we can modify our `get_y` function. First, we define a dictionary for the labels:

In [ ]:
dogs_dict = { 'n02086240': 'Shih-Tzu',
              'n02087394': 'Rhodesian_ridgeback',
              'n02088364': 'beagle',
              'n02089973': 'English_foxhound',
              'n02093754': 'Border_terrier',
              'n02096294': 'Australian_terrier',
              'n02099601': 'golden_retriever',
              'n02105641': 'Old_English_sheepdog',
              'n02111889': 'Samoyed',
              'n02115641': 'dingo',
              }

And define our own `get_y`:

In [ ]:
# function  that retrieves labels from 'dogs_dict' based on the parent directory name of 'x'.
get_y = lambda x: dogs_dict[x.parent.name]

Now, we can create the `DataBlock` again

In [ ]:
woof = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files,
                 splitter=GrandparentSplitter(train_name='train', valid_name='val'),
                 # instead of the parent label, we now use the function from above
                 get_y=get_y,
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75)
                 )

In [ ]:
dls = woof.dataloaders(path)
dls.show_batch()

## Training

Now we will start training our model. We will use a __convolutional neural network__ backbone and a fully connected head with a single hidden layer as a classifier.

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/09/cnn.png?raw=true" style="width:100%; float:left;" />

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/09/convolution.jpg?raw=true" style="width:80%; float:center;" />

### More resources
- For an illustrative explanation of convolutional neural networks (CNNs), see [this video](https://www.youtube.com/watch?v=HGwBXDKFk9I&ab_channel=StatQuestwithJoshStarmer). 
- For a formal coverage, check out the standard book on deep learning [Goodfellow et al (chapter 9)](https://www.academia.edu/download/62266271/Deep_Learning20200303-80130-1s42zvt.pdf), downloadable for free.

### The vision_learner

This method creates a Learner object from the data object and model inferred from it with the backbone given in `arch`.

In [ ]:
learn = vision_learner(dls=dls, arch=models.resnet34, metrics=accuracy)

- __dls__: Dataloaders
- __arch__: architecture.
- __metrics__: accuracy
- __loss_func__: automatically inferred from `dls`. 


### Mentimeter

> What kind of loss function would you typically choose for this task? Answer on [Mentimeter](https://www.menti.com/aleeov3mdgtg).

### Choosing the network architecture

There are many different ways to build a convolutional neural network. For now, the key idea is this: a particular family of models called **ResNets** works well  in most situations.

At this stage, you don’t need to worry about all the architectural details. In practice, your main choice is simply **which size of ResNet to use**. The most common options are:

- **ResNet34** – smaller, faster, and often good enough.
- **ResNet50** – larger, more accurate, but more computationally expensive.

For most tasks early on, deciding between these two is all you really need. (Another choice would be __vision transformers__, but they typically require far more data and compute. ResNets remain widely used because they are simpler, faster, and more reliable on small to medium datasets, making them still an excellent practical default.)


Let's print a summary of the model.

In [ ]:
learn.summary()

__Resnet Architecture__

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/09/resnet-arch.png?raw=true" style="width:70%" />



### Find the learning rate

> [Mentimeter](https://www.menti.com/aleeov3mdgtg): How to identify the learning rate using the learning rate finder?

Please read the fast.ai [lr_finder docs](https://docs.fast.ai/callback.schedule.html#learner.lr_find). Also, Sylvain Gugger from the fast.ai team wrote a nice [blog post](https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html) on how to find  good learning rate.

In [ ]:
learn.lr_find()

The learning rate finder does the following [(adapted from here)](https://sgugger.github.io/how-do-you-find-a-good-learning-rate.html):
* Over an epoch begin your SGD with a very low learning rate (like 10−8) but change it (by multiplying it by a certain factor for instance) at each mini-batch until it reaches a very high value (like 1 or 10).
* Record the loss each time at each iteration and once you're finished, plot those losses against the learning rate. You'll find something like this:

* Plot of the loss against the learning rate: The loss decreases at the beginning, then it stops and it goes back increasing, usually extremely quickly. That's because with very low learning rates, we get better and better, especially since we increase them.
* Then comes a point where we reach a value that's too high and learning process starts to diverge.



Looking at the previous graph, what is the best learning rate to choose?

* __Not the one corresponding to the minimum.__

Why? Well the learning rate that corresponds to the minimum value is already a bit too high, since we are at the edge between improving and getting all over the place. We want to go one order of magnitude before, a value that's still aggressive (so that we train quickly) but still on the safe side from an explosion.

### Fit the model

Fit the model based on selected learning rate

In [ ]:
learn.fit_one_cycle(2, lr_max= 0.001)

We train using the __one cycle policy__. Over the duration of the training process, the policy goes from a lower learning rate to a higher one than going back to the minimum. The maximum learning rate is the value picked with the Learning Rate Finder, and the lower one can be ten times lower. It has been proven to be effective and efficient. See [here](https://sgugger.github.io/the-1cycle-policy.html) for an excellent explanation.

In [ ]:
learn.save('stage-1')

So far we have fitted 2 epochs and it ran pretty quickly. Why is that so? Because we used a little trick (called transfer learning).

What did we do?
We added a few extra layers at the end of architecture, and we only trained those. We left most of the early layers as they were. This is called freezing layers i.e weights of the layers.

- When we call fit or `fit_one_cycle()` on a create_cnn, it will just fine-tune these extra layers at the end, and run very fast.
- To get a better model, we have to call `unfreeze()` to train the whole model.

### Unfreeze and train again

Since our model is working as we expect it to, we will unfreeze our model and train some more.

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

__Learning rates after unfreezing__

The basic rule of thumb is after you unfreeze (i.e. train the whole thing), pass a max learning rate parameter, pass it a slice, make the second part of that slice about 10 times smaller than your first stage.

In [ ]:
learn.fit_one_cycle(10, lr_max=slice(1e-5,1e-4))

### Mentimeter

> Which next steps would you undertake based on these learning results? Answer on [Mentimeter](https://www.menti.com/aleeov3mdgtg).

- **Need for Different Learning Rates:**
  - Early layers (from pre-trained model) need less adjustment.
  - Later layers (trained for specific task) require more significant learning.
  - High learning rates on early layers can disrupt pre-learned features.

- **The Rule of Thumb:**
  - Use differential learning rates after unfreezing the model.
  - __Lower__ rate for __early__ layers, __higher__ rate for __later__ layers.
  - In FastAI, use a "slice" to specify rates: later layers' rate ~10x the early layers'.

- **Fine-Tuning and Stability:**
  - Approach ensures stable, effective fine-tuning across the model.
  - Early layers receive small updates to adjust learned features.
  - Later layers undergo more rapid learning with task-specific data.

If the model overfits we can reload stage 1 and train the model again for fewer epochs or another learning rate.

In [ ]:
# Loads the model state previously saved under the name 'stage-1'.
learn = learn.load('stage-1')
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(3, lr_max=slice(1e-5,1e-4))

## Results

It’s important to see what comes out of our model. We have seen one way of what goes in, now let’s see what our model has predicted.

The `ClassificationInterpretation` class has methods for creating confusion matrix as well as plotting misclassified images.

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

### Top Losses

We will first see which were the categories that the model most confused with one another. We will try to see if what the model predicted was reasonable or not.

In [ ]:
interp.plot_top_losses(9, largest=True, figsize=(18,18))

### Confusion Matrix

Furthermore, when we plot the confusion matrix. Interestingly, the model often confuses English Foxhounds with Beagles. This confirmes that our model works very well until a certain level as these two dog breeds look very similar.

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

__Most Confused__

Sorted descending list of largest non-diagonal entries of confusion matrix, presented as actual, predicted, number of occurrences.

In [ ]:
 interp.most_confused(min_val=2)

## Exercises: Explainability of The Model using Grad-CAM

After training a model, you obtain the classification results of given images. Have you ever wondered which part of an image the model focuses on when making a prediction about its class? In this exercise, you have a chance to investigate [Grad-CAM](https://arxiv.org/abs/1610.02391), a visualization technique that helps to look into the captured features of individual layers of a convolutional neural nework.



## What is Grad-CAM?

Gradient-weighted Class Activation Mapping (Grad-CAM) is a technique used to visualize and understand what parts of an image a deep learning model focuses on when making predictions. It works by computing the gradients of the target class score with respect to the feature maps of a specific convolutional layer. These gradients highlight the importance of each region in the feature maps, which are then combined and overlaid onto the original image to produce a heatmap. This heatmap shows which regions of the image contributed the most to the model's decision, making it easier to interpret and debug the model's behavior.

In simple words, Grad-CAM helps us understand a model's decision-making process by highlighting which features in specific layers contribute most to a prediction. To use it effectively, we typically focus on deeper convolutional layers where high-level features are learned, and analyze how these features correlate with the model's output. This insight can guide debugging, model refinement, or identifying biases in the predictions. In this exercise, we will take a look at the final convolutional layer of the `resnet34` model.

## Task 1: Identify the final convolutional layer of ResNet34

After training the model as instructed in the lecture, we want to take a look at the feature maps in the final block of convolutional layers. The goal of Grad-CAM is to take the output feature maps from the last CNN block and figure out which parts are most important for predicting a specific class. To do this, we calculate the average gradient of each feature map with respect to the class score, which gives us their importance. Then, we use these weights to combine the feature maps, highlighting the areas in the image that influenced the prediction.









Take a look at the model summary again:

In [ ]:
learn.summary()

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/09/resnet-arch.png?raw=true" style="width:70%" />

Now, take out the final convolutional block.

Hint 1: It should be in the form of `learn.model[0][7][final_block]`

Hint 2: First index [0] is to access to the CNN layers. It is built based on a `Sequential` class in `PyTorch`. You can print `learn.model` to see the structure of the model. `learn.model[1]` allows you to access to the linear layers (the avg pool + white block `fc 1000` in the figure.

Hint 3: Second index is to access to the second nested `Sequential` block. You can find the required index by printing `learn.model[0][7]`.

In [ ]:
# Your Your Code Here
last_cnn_block = 

We create the heatmap by linearly combining the feature maps, inspired by a [scaled-down](https://github.com/henripal/maps/blob/master/nbs/big_resnet50-interpret-gradcam-dogs.ipynb) version of the gradients as weights.In FastAI, a [Hook](https://docs.fast.ai/callback.hook.html) is a utility that allows you to "hook into" the internal workings of a model during the forward or backward pass. It lets you capture or modify the intermediate outputs, gradients, or other data flowing through specific layers of a model.


In [ ]:
# ---- No need to touch this code, only run it ----
import fastai

# Hook for the feature maps
fmap_hook = Hook(last_cnn_block, lambda m, i, o: o)

# Hook for the gradients
def gradient_torch_hook(module, grad_input, grad_output):
    return grad_output[0]

gradient_hook = Hook(last_cnn_block, gradient_torch_hook, is_forward=False)

# Convert image tensor to numpy for visualization
def image_from_tensor(imagetensor):
    numpied = imagetensor.squeeze().permute(1, 2, 0).cpu().numpy()
    numpied -= numpied.min()
    numpied /= numpied.max()
    return numpied

## 2. Task 2: Extract images from the first batch  

First, let's load the first batch of images:

In [ ]:
# Your Code Here
# Hint: use first() from fastai
image_batch, label_batch = 

Next, set the model into evaluation mode because we are only interested in the forward-pass computation:

In [ ]:
# Your Code Here
# Hint 1: set `learn' model to eval() mode
# Hint 2: check out https://docs.fast.ai/tutorial.imagenette.html#training


Finally, we compute the gradients and visualize several images. Your final task is to fill in 3 lines of code:
* set up the for loop with `16` images,
* set the model into `zero_grad()` [mode](https://datascience.stackexchange.com/questions/124487/what-do-we-mean-by-optimizer-zero-grad#:~:text=We%20need%20to%20invoke%20zero_grad,gradients%20to%20update%20the%20weights.), and
* perform a `forward pass`.

We’ve introduced deep learning at a high level, but now you’re seeing how to interact with the model at a deeper level by exploring the forward and backward passes. If you’d like to learn more about how a training pipeline works in PyTorch, check out this [guide](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html).

In [ ]:
import cv2

# Setup for visualization
fig, axes = plt.subplots(4, 4, figsize=(12, 12))
fig.tight_layout()

# [1] Your Code Here
# Define 16 images to use
for i in range(16):
    image =   # Add batch dimension
    label =   # Add batch dimension

    # [2] Your Code Here
    # Perform zero_grad here
    # Hint: check out https://docs.fast.ai/tutorial.imagenette.html#training
    

    # [3] Your Code Here
    # Run the forward pass here using "image" defined above
    # Hint: call learn.model(input)
    out = 

    # Create one-hot encoding for the predicted class
    onehot = torch.zeros_like(out)
    onehot[0, torch.argmax(out)] = 1.0

    # Backpropagate from the one-hot encoded output
    out.backward(gradient=onehot, retain_graph=True)

    # Retrieve gradients and feature maps
    gradients = gradient_hook.stored
    gradients = gradients[0].detach().cpu().numpy()
    fmaps = fmap_hook.stored
    fmaps = fmaps[0].detach().cpu().numpy()

    # Process gradients and feature maps
    gradient_linearization = gradients.sum((1, 2))  # Sum across width and height
    weighted_fmaps = (fmaps * gradient_linearization[:, None, None]).sum(0)

    # Normalize heatmap
    heatmap = np.maximum(weighted_fmaps, 0)
    heatmap /= heatmap.max()

    # Resize heatmap to match input image size
    upsampled = cv2.resize(heatmap, (image.shape[2], image.shape[3]))

    # Display the image and heatmap
    ax = axes[i // 4, i % 4]
    ax.imshow(image_from_tensor(image[0]))
    ax.imshow(upsampled, alpha=0.6, cmap='jet')
    ax.axis('off')

plt.show()

That's it! If you are interested in more fine-grained heatmaps, consider adapting this [code](https://github.com/henripal/maps/blob/master/nbs/big_resnet50-interpret-gradcam-dogs.ipynb) into this exercise! You might have to adjust the code from `fastaiv1` to `fastaiv2` to make it compatible to this notebook's code.

<img src="https://github.com/GuntherGust/tds2_data/blob/main/images/d3.png?raw=true" style="width:50%; float:center;" />